In [ ]:
from lxml import html
import json
import requests


## 1.1.1 - AWS Unique Identifiers
apparel = {
    "women": {
        "tops" : ["B075FLTQYL", "B01J7QP7QY", "B06XBJBGHF", "B073H2XXZZ", "B07571M8QP", "B00ZUNH1V8"],
        "bottoms" : ["B071KCST2M", "B01DTZV3GC", "B00B5W8310", "B01DVR2X76", "B01MQWGKL0", "B06XRXZCK4", "B01MYDQV2P"],
        "accessories" : ["B01LWUNIPA","B074HXSLBH", "B072C8G68X", "B075QVBTNJ", "B0077G0VZE", "B075MX1TV9", "B00GBCTSYU", "B01N8PBVGL", "B0748BKXJB"]
    },
    "men" :{
        "tops" : ["B06X3SGD2B", "B01B3GMNU4", "B00TUMS344", "B00GOZPM44", "B072MS3WW5", "B073QGVRBH", "B0741C4QJ3"],
        "bottoms" : ["B073XD9NH2"],
        "accessories" : ["B072C8G68X", "B0748BKXJB", "B01N8PBVGL"]
    }
}

## 1.1.2 - Shorthand code for products
m_t = apparel["men"]["tops"]
w_t = apparel["women"]["tops"]
m_b = apparel["men"]["bottoms"]
w_b = apparel["women"]["bottoms"]
m_a = apparel["men"]["accessories"]
w_a = apparel["women"]["accessories"]
## need to concatenate lists for unisex
u_t = m_t + w_t
u_b = m_b + w_b
u_a = m_a + w_a

## remove duplicates from m/w
u_t = list(set(u_t))
u_b = list(set(u_b))
u_a = list(set(u_a))


## 2.1.1 - take category, return list of urls
def getUrls(x):
    urls = []
    for index, item in enumerate(x):
        url = "https://www.amazon.com/gp/product/" + item
        urls.append(url)
    return(urls)

## 2.1.2 - input url, output data dict

def parse(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    page = requests.get(url,headers=headers)
    for i in range(20):
        try:
            doc = html.fromstring(page.content)
            XPATH_NAME = '//h1[@id="title"]//text()'
            XPATH_SALE_PRICE = '//span[contains(@id,"ourprice")]/text()'
            XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()'
            ## pull image
            XPATH_IMAGE = '//img[@id="landingImage"]'
            
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            
            RAW_IMAGE = doc.xpath(XPATH_IMAGE)

            for link_element in RAW_IMAGE:
                src = link_element.get('src')
                       
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None

            IMAGE = RAW_IMAGE if RAW_IMAGE else None
            
## need to decode this from base 64.
## https://stackoverflow.com/questions/42834675/base64-string-to-image-in-tkinter

           
            if not ORIGINAL_PRICE:
                ORIGINAL_PRICE = SALE_PRICE
                
            # retrying in case of caotcha
            if not NAME :
                raise ValueError('captcha')
                
            # output
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'IMAGE':IMAGE
                    }

            return data
        except Exception as e:
            print (e)

## 2.2.1 - input category, send to getUrls(x)
##         append each url in getUrls(x) as search
##         output results
def search(x):
    ## 2.2.2 - conditional formatting based on category  
    if x == w_t:
        print("Searching For Women's Tops")
    if x == w_b:
        print("Searching For Women's Bottoms")
    if x == w_a:
        print("Searching For Women's Accessories")
    if x == m_t:
        print("Searching For Men's Tops")
    if x == m_b:
        print("Searching For Men's Bottoms")
    if x == m_a:
        print("Searching For Men's Accessories")
    if x == u_t:
        print("Searching For Tops")
    if x == u_b:
        print("Searching For Bottoms")
    if x == u_a:
        print("Searching For Accessories")
    results = []
    for index, item in enumerate(getUrls(x)):
        product = parse(item)
        results.append(product)
    return(results)

def extractData(x):
    n = []
    for index, item in enumerate(x):
        n.append(item["NAME"])
        n.append(item["SALE_PRICE"])
        n.append(item["CATEGORY"])
        n.append(item["ORIGINAL_PRICE"])
        n.append(item["AVAILABILITY"])
        n.append(item["URL"])
        n.append(item["IMAGE"])
        
## final result - print out name, sale price, url and categories
    print(n)
        

    

from tkinter import *
root = Tk()

def mT(event):
    mT = search(m_t) ## loads up the search
    extractData(mT) ## executes the command

def wT(event):
    wT = search(w_t)
    extractData(wT)
        
def uT(event):
    uT = search(u_t)
    extractData(uT)
        
def mB(event):
    mB = search(m_b)
    extractData(mB)

def wB(event):
    wB = search(w_b)
    extractData(wB)
        
def uB(event):
    uB = search(u_b)
    extractData(uB)
        
def mA(event):
    mA = search(m_a)
    extractData(mA)

def wA(event):
    wA = search(w_a)
    extractData(wA)
        
def uA(event):
    uA = search(u_a)
    extractData(uA)
        
topFrame = Frame(root)
bottomFrame = Frame(root)
resultsFrame = Frame(root)
topFrame.pack(side=LEFT)
bottomFrame.pack(side=LEFT)
resultsFrame.pack(side=BOTTOM)
masthead = Label(topFrame, text="ShoppingList", bg="black", fg="white")
masthead.pack()

m_tops = Button(bottomFrame, text="Men's Tops")
m_tops.bind("<Button-1>", mT)
m_tops.grid(row="0")

w_tops = Button(bottomFrame, text="Women's Tops")
w_tops.bind("<Button-1>", wT)
w_tops.grid(row="0", column="1")
    
u_tops = Button(bottomFrame, text="All Tops")
u_tops.bind("<Button-1>", uT)
u_tops.grid(row="0", column="2")
    
m_bottoms = Button(bottomFrame, text="Men's Bottoms")
m_bottoms.bind("<Button-1>", mB)
m_bottoms.grid(row="1")

w_bottoms = Button(bottomFrame, text="Women's Bottoms")
w_bottoms.bind("<Button-1>", wB)
w_bottoms.grid(row="1", column="1")
    
u_bottoms = Button(bottomFrame, text="All Bottoms")
u_bottoms.bind("<Button-1>", uB)
u_bottoms.grid(row="1", column="2")
    
m_accessories = Button(bottomFrame, text="Men's Accessories")
m_accessories.bind("<Button-1>", mA)
m_accessories.grid(row="2")

w_accessories = Button(bottomFrame, text="Women's Accessories")
w_accessories.bind("<Button-1>", wA)
w_accessories.grid(row="2", column="1")
    
u_accessories = Button(bottomFrame, text="All Accessories")
u_accessories.bind("<Button-1>", uA)
u_accessories.grid(row="2", column="2")
    
root.mainloop()

Searching For Women's Tops
['Youtalia Womens 3/4 Cuffed Sleeve Chiffon Printed V Neck Casual Blouse Shirt Tops', None, 'Clothing, Shoes & Jewelry > Women > Clothing > Tops & Tees > Blouses & Button-Down Shirts', None, None, 'https://www.amazon.com/gp/product/B075FLTQYL', [<Element img at 0x2a96bf8db88>]]
['Youtalia Womens 3/4 Cuffed Sleeve Chiffon Printed V Neck Casual Blouse Shirt Tops', None, 'Clothing, Shoes & Jewelry > Women > Clothing > Tops & Tees > Blouses & Button-Down Shirts', None, None, 'https://www.amazon.com/gp/product/B075FLTQYL', [<Element img at 0x2a96bf8db88>], "LOSRLY Women's Loose Solid Chiffon Blouses V Neck Cuffed Sleeve Shirts Tops", None, 'Clothing, Shoes & Jewelry > Women > Clothing > Tops & Tees > Blouses & Button-Down Shirts', None, None, 'https://www.amazon.com/gp/product/B01J7QP7QY', [<Element img at 0x2a96bf8dcc8>]]
['Youtalia Womens 3/4 Cuffed Sleeve Chiffon Printed V Neck Casual Blouse Shirt Tops', None, 'Clothing, Shoes & Jewelry > Women > Clothing > Top

In [38]:
## parsing html from example search using regex
## import re
## pattern = re.compile(r"http://www.amazon.com/.*/dp/(.*?)\"")
## re.findall(pattern)